In [7]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())

dict_keys([])


In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('/content/filtered_data(저가형)(찐최종).csv')

In [10]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,lcom_count,ccom_count,lcafe_count,hcafe_count,centroid,density,density_rank
0,0,0,0,8,0,0,0,0,0,0,POINT (127.071586 37.429794),0,0
1,0,0,0,0,0,0,0,0,1,0,POINT (126.900586 37.432793999999994),1,0
2,0,0,0,4,0,0,0,0,0,0,POINT (126.90358600000002 37.432793999999994),0,0
3,0,0,1,0,0,0,0,0,0,0,POINT (127.12858600000001 37.432793999999994),0,0
4,0,0,0,0,0,0,0,0,0,1,POINT (126.900586 37.435794),0,0


In [11]:
# 1. 'centroid' 열 삭제
df = df.drop(columns=['centroid'])

# 2. 'rank' 열의 항목과 각 항목의 갯수 확인
rank_counts = df['density_rank'].value_counts()

print(df)  # 업데이트된 DataFrame 출력
print(rank_counts)  # rank 열의 항목들과 갯수 출력

      school_count  academy_count  subway_count  bus_count  apt_count  \
0                0              0             0          8          0   
1                0              0             0          0          0   
2                0              0             0          4          0   
3                0              0             1          0          0   
4                0              0             0          0          0   
...            ...            ...           ...        ...        ...   
4590             0              0             0         20          0   
4591             0              0             0          4          0   
4592             0              0             0          4          0   
4593             1              0             0          0          0   
4594             0              0             1          0          0   

      univ_count  lcom_count  ccom_count  lcafe_count  hcafe_count  density  \
0              0           0           0    

In [12]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,lcom_count,ccom_count,lcafe_count,hcafe_count,density,density_rank
0,0,0,0,8,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,1,0
2,0,0,0,4,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0


In [13]:
# 1. 'density' 열 삭제
df = df.drop(columns=['density'])

In [14]:
df['company'] = df['lcom_count'] + df['ccom_count']
df = df.drop(columns=['lcom_count'])
df = df.drop(columns=['ccom_count'])

In [15]:
df = df.drop(columns=['lcafe_count'])

In [16]:
df = df.drop(columns=['hcafe_count'])

In [17]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,density_rank,company
0,0,0,0,8,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,4,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0


In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
data = df[['school_count', 'academy_count', 'bus_count','subway_count', 'apt_count', 'univ_count','company']].to_numpy()
target = df['density_rank'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data ,target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# CPU를 모두 이용해 병렬로 교차 검증 수행
# return_train_score=True: 훈련 세트에 대한 점수도 같이 반환합니다. => 과대적합 파악에 유용
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.9973541965122431 0.8905151032797809

0.9821137472329973 0.971164596273292


In [19]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것.
# 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문에 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. => 과대적합 줄일 수o

[0.08448221 0.47887301 0.26920763 0.02840594 0.10514498 0.00178543
 0.03210081]


In [20]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
# oob_score의 기본은 false입니다.
# oob: 부트스트랩 샘플에 포함되지 않고 남은 샘플.
rf.fit(train_input, train_target)
print(rf.oob_score_)
# 0.8934000384837406
# oob로 자체적 평가 => 교차 검증을 대신할 수 있어 훈련 세트에 더 많은 샘플을 이용할 수 있다.

0.970892274211099


In [21]:
# 모델 훈련
rf.fit(train_input, train_target)

# 각 트리의 깊이를 리스트로 저장
tree_depths = [estimator.tree_.max_depth for estimator in rf.estimators_]

# 랜덤 포레스트 내 트리들의 최대 깊이 출력
max_tree_depth = max(tree_depths)
print("랜덤 포레스트 내 트리들의 최대 깊이:", max_tree_depth)

# 모든 트리의 깊이 출력 (선택사항)
print("모든 트리들의 깊이:", tree_depths)


랜덤 포레스트 내 트리들의 최대 깊이: 26
모든 트리들의 깊이: [23, 19, 17, 16, 16, 20, 24, 19, 18, 19, 20, 20, 23, 20, 18, 20, 24, 20, 22, 20, 21, 21, 17, 26, 18, 22, 19, 20, 18, 20, 19, 20, 22, 17, 22, 18, 17, 19, 22, 20, 19, 19, 21, 17, 19, 20, 22, 16, 20, 17, 19, 23, 22, 22, 19, 23, 18, 19, 18, 19, 21, 18, 21, 19, 19, 17, 17, 18, 20, 17, 19, 21, 20, 20, 19, 22, 21, 17, 17, 20, 21, 22, 18, 21, 20, 19, 21, 18, 17, 15, 19, 20, 19, 18, 20, 21, 19, 20, 20, 18]


In [23]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 'max_depth': range(3, 5, 1), 'min_samples_split': range(2, 100, 10)}
# arange(): 첫 번째 매개변수 값에서 시작하여 두 번째 매개변수에 도달할 때까지 세 번째 매개변수를 계속 더한 배열을 만든다.
# range(): (정수만 사용) 위의 함수와 동일
# 따라서 9 x 10 x 10 = 1350개의 모델을 만든다. 기본 5-폴드 교차를 수행하니 모델의 수는 6750개이다.

gs = GridSearchCV(RandomForestClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

print(gs.best_params_)
# {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

print(np.max(gs.cv_results_['mean_test_score']))
# 0.8683865773302731

{'max_depth': 3, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}
0.9741566844128956


range(5,20,1) : {'max_depth': 5, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}
0.9741566844128956 <br/>
range(3,5,1) : {'max_depth': 3, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}
0.9741566844128956

최적의 training score, test score 출력

In [24]:

data = df[['school_count', 'academy_count', 'bus_count','subway_count', 'apt_count', 'univ_count','company']].to_numpy()
target = df['density_rank'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data ,target, test_size=0.2, random_state=42)

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=3, min_impurity_decrease=0.0001, min_samples_split=2, n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
# CPU를 모두 이용해 병렬로 교차 검증 수행
# return_train_score=True: 훈련 세트에 대한 점수도 같이 반환합니다. => 과대적합 파악에 유용
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 0.9973541965122431 0.8905151032797809

0.9741566915783654 0.9741566844128956


In [25]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.05573432 0.45633701 0.2260507  0.05896648 0.12715418 0.
 0.07575731]


In [26]:
df.head()

,school_count,academy_count,subway_count,bus_count,apt_count,univ_count,density_rank,company
0,0,0,0,8,0,0,0,0
1,0,0,0,0,0,0,0,0
2,0,0,0,4,0,0,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0
